在開始之前, 先從 Kaggle 下載訓練以及測試用的資料:

```
kg download -u `user_name` -p `password` -c dogs-vs-cats-redux-kernels-edition
```

不過在下載之前, 必須先到 Kaggle 註冊帳號, 以及同意 Competition 的規則才能下載檔案:
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition

Todo:
1. 設立 Sample 與 Valid data set 目錄
2. 將 Kaggle 的檔案放置到符合 Keras 的目錄結構下
3. 載入 VGG16 model, finetune 以及重新對 dogs & cats 作訓練
4. 預測 Kaggle 的 test set
5. 驗證測試結果
6. 在 Kaggle 上送交結果

## 設立 Sample 與 Valid data set 目錄

In [1]:
%pwd
%matplotlib inline

In [2]:
import os, sys
current_dir = os.getcwd()
ROOT_PATH = current_dir
DATA_HOME_PATH = current_dir + '/data/redux'
DATA_HOME_PATH

'/home/paperspace/src/fastai/deeplearning_keras2/steven/data/redux'

In [3]:
import numpy as np
from glob import glob
from shutil import copyfile

In [ ]:
%cd $DATA_HOME_PATH
%mkdir valid
%mkdir results
%mkdir sample
%mkdir -p sample/valid
%mkdir -p sample/train

In [4]:
# 準備驗證資料

%cd $DATA_HOME_PATH/train
all_training_files = glob("*.jpg")

# 打亂檔案列表
shuffles = np.random.permutation(all_training_files)
if len(shuffles) > 0:
    # 取其中 2000 個檔案作為驗證資料用
    for i in range(0, 2000):
        os.rename(shuffles[i], DATA_HOME_PATH + '/valid/' + shuffles[i])

/home/paperspace/src/fastai/deeplearning_keras2/steven/data/redux/train


In [ ]:
# 為解省開發上時間的耗費, 會建立一個資料量相對小的 Sample 目錄, 程式開發完之後再轉移到完整的資料上
# 準備 Sample 的 training 資料
%cd $DATA_HOME_PATH/train
all_training_files = glob("*.jpg")

# 打亂檔案列表
shuffles = np.random.permutation(all_training_files)
if len(shuffles) > 0:
    # 取其中 200 個作為 Sample 的訓練資料
    for i in range(0, 200):
        copyfile(shuffles[i], DATA_HOME_PATH + '/sample/train/' + shuffles[i])

In [ ]:
# 準備 Sample 的 Valid 資料
%cd $DATA_HOME_PATH/valid

all_valid_files = glob("*.jpg")
shuffles = np.random.permutation(all_valid_files)
if len(shuffles) > 0:
    # 拿其中 50 個檔案作為 Sample 的驗證資料
    for i in range(0, 50):
        copyfile(shuffles[i], DATA_HOME_PATH + '/sample/valid/' + shuffles[i])

## 將 Kaggle 的檔案放置到符合 Keras 的目錄結構下

Keras 的目錄結構用「類別」名稱作來命名子目錄, 從 Kaggle 下載下來的檔案則是用檔名的區分, 例如 cats.3111.jpg, 所以在這個步驟我們要建立 cats 跟 dogs 子目錄, 並將這些圖檔搬移到相對應的子目錄裡

In [ ]:
%cd $DATA_HOME_PATH/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_PATH/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_PATH/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_HOME_PATH/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

In [ ]:
%cd $DATA_HOME_PATH/test
%mkdir unknown
%mv *.jpg unknown/

## 載入 VGG16 model, finetune 以及重新對 dogs & cats 作訓練

VGG16 是 Visual Geometry Group 的縮寫, 通常有分作 16 層跟 19 層 Neuon Network 的兩種版本, 它可以辨識 ImageNet 中 1500 個影像類別, 是個十分強大的 CNN 演算法, 網路上也可以下載到 pre-trained 的 model, 省下一開始找資料以及 training 上的時間, 可以直接拿來應用

這裡用的 VGG16 是直接拿 fast.ai 的實作版本, 這個實作版本與 fast.ai github 上的版本不同, 採用 Python3 跟 Keras2.0

In [5]:
%cd $ROOT_PATH

from vgg16 import Vgg16

path = DATA_HOME_PATH + '/sample/'
valid_path = path + '/valid/'
train_path = path + '/train/'
test_path = DATA_HOME_PATH + '/test/'
result_path = DATA_HOME_PATH + '/results/'

/home/paperspace/src/fastai/deeplearning_keras2/steven


Using TensorFlow backend.


In [6]:
# 初始化 vgg 物件, 第一次初始會下載 Vgg16 pre-trained 的 weights, 下載檔案會放在 ~/.keras/models/
vgg = Vgg16()

In [7]:
batch_size = 4
epoch_num = 10

原本 Vgg16 model 可以偵測 1500 種類別, 但是在這裡我們只有兩種類別, 所以透過 Keras 的 finetune 機制將原本的 model mapping 到這兩種類別上

get_batches 會使用 Keras API - [Image Preprocessing](https://keras.io/preprocessing/image/), 從指定的目錄中批次將圖片讀出, 並對圖片作正規化, 每張圖片縮放成 244x244 大小

In [8]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
vgg.finetune(batches)

Found 200 images belonging to 2 classes.


接下來, 我們可以跑幾個 epoch 來 retrain model

In [9]:
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)
vgg.fit(batches, val_batches, batch_size, nb_epoch=epoch_num)
latest_weights_filename = 'ft%d.h5'
vgg.model.save_weights(result_path + latest_weights_filename)

Found 50 images belonging to 2 classes.
Epoch 1/10
50/50 [==============================] - 9s - loss: 0.7724 - acc: 0.7600 - val_loss: 0.7335 - val_acc: 0.7600
Epoch 2/10
50/50 [==============================] - 5s - loss: 0.3012 - acc: 0.8950 - val_loss: 0.6426 - val_acc: 0.7800
Epoch 3/10
50/50 [==============================] - 5s - loss: 0.3682 - acc: 0.9250 - val_loss: 0.6923 - val_acc: 0.8200
Epoch 4/10
50/50 [==============================] - 5s - loss: 0.1478 - acc: 0.9400 - val_loss: 0.6790 - val_acc: 0.8400
Epoch 5/10
50/50 [==============================] - 5s - loss: 0.1331 - acc: 0.9400 - val_loss: 0.7952 - val_acc: 0.8600
Epoch 6/10
50/50 [==============================] - 5s - loss: 0.1250 - acc: 0.9400 - val_loss: 0.4921 - val_acc: 0.8400
Epoch 7/10
50/50 [==============================] - 5s - loss: 0.1255 - acc: 0.9600 - val_loss: 0.8544 - val_acc: 0.7600
Epoch 8/10
50/50 [==============================] - 5s - loss: 0.1243 - acc: 0.9450 - val_loss: 0.7495 - val_acc:

## 測試

In [ ]:
batches, preds = vgg.test(test_path, batch_size=batch_size)
preds[:3]

Found 12500 images belonging to 1 classes.
